In [1]:
import pandas as pd
import numpy as np
import pandas_gbq
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV, KFold
from google.cloud import storage
import os

In [68]:
d.pop('car_attributes', None)

'<webapp_platform.models.CarAttributes object at 0x7fd625809c50>'

In [8]:
query = """
SELECT
  *
FROM
  car-platform-273211.car_listings.car_listings
"""
df_raw = pd.read_gbq(query,
                     project_id = os.environ['GCP_PROJECT_ID'],
                     dialect = 'standard')

In [151]:
df = df_raw.copy()

In [154]:
df.columns

Index(['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer',
       'model', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status',
       'transmission', 'vin', 'drive', 'size', 'type', 'paint_color',
       'image_url', 'description', 'county', 'state', 'lat', 'long'],
      dtype='object')

In [ ]:
### cheat

In [156]:
df = df[['year', 'odometer', 'price']]

In [157]:
df.shape

(539759, 3)

In [158]:
df.isnull().sum()

year          987
odometer    98976
price           0
dtype: int64

In [159]:
df = df.dropna()
df.shape

(439815, 3)

In [160]:
df.rename(columns = {"price": "customer_price", "odometer": "mileage"}, inplace = True)

/Users/marichka/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4304: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [161]:
df.head()

,year,mileage,customer_price
2,2011.0,192904.0,5780
3,2001.0,71000.0,7900
4,2017.0,46543.0,16600
6,2003.0,161000.0,3495
7,2010.0,43658.0,17900


In [ ]:
### not cheat

In [96]:
df.shape

(539759, 25)

In [97]:
df.type.value_counts()

sedan          96119
SUV            94626
pickup         55934
truck          50419
coupe          21664
other          18351
hatchback      14791
wagon          12194
van            11006
convertible     9089
mini-van        6685
offroad          835
bus              577
Name: type, dtype: int64

In [102]:
df.model.value_counts()[:15]

f-150             11630
                   7978
silverado 1500     7302
1500               6623
silverado          5679
wrangler           4270
accord             3933
altima             3920
2500               3865
camry              3746
grand cherokee     3637
escape             3590
civic              3398
explorer           3368
tacoma             3203
Name: model, dtype: int64

In [103]:
df = df[(df.type == 'hatchback') | (df.type == 'pickup') | (df.type == 'sedan') | (df.type == 'wagon') | 
  (df.type == 'van') | (df.type == 'truck')]

In [104]:
df = df[(df.state == 'mn') | (df.state == 'va') | (df.state == 'wi') | (df.state == 'ca') | (df.state == 'fl') | 
       (df.state == 'wa') | (df.state == 'tx') | (df.state == 'ny')]

In [105]:
df = df[(df.fuel == 'gas') | (df.fuel == 'diesel')]

In [106]:
df = df[(df.transmission == 'automatic') | (df.transmission == 'manual') | (df.transmission == 'other')]

In [107]:
df = df[(df.model == 'f-150') | (df.model == 'camry') | (df.model == 'civic') | (df.model == 'tacoma') | 
       (df.model == 'silverado')]

In [132]:
df = df[df.color != 'custom']

In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6853 entries, 18 to 539741
Data columns (total 25 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            6853 non-null   int64  
 1   url           6853 non-null   object 
 2   region        6853 non-null   object 
 3   region_url    6853 non-null   object 
 4   price         6853 non-null   int64  
 5   year          6853 non-null   float64
 6   manufacturer  6852 non-null   object 
 7   model         6853 non-null   object 
 8   condition     3608 non-null   object 
 9   cylinders     4641 non-null   object 
 10  fuel          6853 non-null   object 
 11  odometer      6163 non-null   float64
 12  title_status  6841 non-null   object 
 13  transmission  6853 non-null   object 
 14  vin           4538 non-null   object 
 15  drive         6312 non-null   object 
 16  size          3124 non-null   object 
 17  type          6853 non-null   object 
 18  paint_color   5715 non-nu

In [110]:
df.describe()

,id,price,year,odometer,lat,long
count,6.853000e+03,6.853000e+03,6853.000000,6.163000e+03,6796.000000,6796.000000
mean,7.087674e+09,1.528763e+04,2010.684518,1.054187e+05,37.127656,-99.643729
std,5.375831e+06,1.344977e+05,6.245581,7.583317e+04,6.801116,17.983059
min,7.074439e+09,0.000000e+00,1973.000000,0.000000e+00,-67.501400,-124.249000
25%,7.084084e+09,4.995000e+03,2007.000000,5.371700e+04,31.477125,-119.789000
50%,7.089132e+09,1.199500e+04,2012.000000,1.000000e+05,37.623700,-97.262600
75%,7.092369e+09,2.049000e+04,2015.000000,1.451420e+05,42.902450,-81.738800
max,7.093534e+09,1.111111e+07,2020.000000,2.799000e+06,51.183200,56.213200


In [111]:
df.isnull().sum()

id                 0
url                0
region             0
region_url         0
price              0
year               0
manufacturer       1
model              0
condition       3245
cylinders       2212
fuel               0
odometer         690
title_status      12
transmission       0
vin             2315
drive            541
size            3729
type               0
paint_color     1138
image_url          0
description        0
county          6853
state              0
lat               57
long              57
dtype: int64

In [112]:
df.shape

(6853, 25)

In [113]:
df.vin.value_counts()

5TFLU4ENXCX041079    20
3TMCZ5AN7JM138269    19
3TMDZ5BN4GM010566    19
5TEMU52N26Z150728    19
1FTFW1EF0EKE49282    16
                     ..
19XFB2F52CE060232     1
1FTNW21FX3EA62306     1
1FTEW1CF6HKE48933     1
1FTFW1ET2EFB64971     1
1GC4KYC86FF616447     1
Name: vin, Length: 2734, dtype: int64

In [114]:
df.drive.value_counts()

4wd    3299
fwd    1865
rwd    1148
Name: drive, dtype: int64

In [115]:
df['size'].value_counts()

full-size      2217
mid-size        513
compact         347
sub-compact      47
Name: size, dtype: int64

In [116]:
df['type'].value_counts()

pickup       2875
truck        2242
sedan        1681
hatchback      50
van             4
wagon           1
Name: type, dtype: int64

#### Dealing with missing values

In [117]:
df = df.drop(columns = 'county') # country column is NaN?

In [118]:
df.drop(columns = ['id', 'url', 'region_url', 'vin', 'image_url', 'region', 'lat', 'long'], inplace = True)

In [119]:
# color_list = list(df.paint_color.unique())
# color_list.remove(None)

In [120]:
# color_list

In [121]:
# def check_for_color(x):
#     if x is not None:
#         for i in color_list:
#             if i in x.lower().split():
#                 return i

In [122]:
# df['color'] = df['description'].apply(check_for_color)

In [123]:
# df.color.value_counts()

In [124]:
# df['color'] = np.where(df['paint_color']== None, df['color'], df['paint_color'])

In [125]:
df.isnull().sum()

price              0
year               0
manufacturer       1
model              0
condition       3245
cylinders       2212
fuel               0
odometer         690
title_status      12
transmission       0
drive            541
size            3729
type               0
paint_color     1138
description        0
state              0
dtype: int64

In [126]:
df.drop(columns = ['condition', 'cylinders', 'title_status', 'drive', 'size', 'description'], axis = 1, 
        inplace = True)

In [127]:
df = df.dropna()

In [128]:
df.shape

(5131, 10)

In [129]:
df.rename(columns = {"price": "customer_price", "manufacturer": "manufacturer_id", 
                  "model": "model_id", "odometer": "mileage", "paint_color": "color"}, inplace = True)

In [130]:
df.head()

,customer_price,year,manufacturer_id,model_id,fuel,mileage,transmission,type,color,state
18,1,2011.0,toyota,tacoma,gas,168955.0,automatic,truck,silver,tx
141,20988,2011.0,chevrolet,silverado,gas,82000.0,automatic,truck,white,tx
221,13699,2004.0,chevrolet,silverado,gas,135101.0,automatic,truck,green,va
520,20818,2013.0,ford,f-150,gas,50278.0,automatic,truck,red,wi
642,4800,1997.0,ford,f-150,gas,111000.0,automatic,truck,white,tx


In [133]:
# def cylinder_transformation(x):
#     if x != 'other':
#         return int(x.split()[0])
#     elif x == 'other':
#         return 'other'

# df['cylinders'] = df['cylinders'].apply(cylinder_transformation)

In [134]:
df = pd.get_dummies(df)

In [135]:
df.shape

(5010, 43)

In [136]:
df.head()

,customer_price,year,mileage,manufacturer_id_chevrolet,manufacturer_id_dodge,manufacturer_id_ford,manufacturer_id_honda,manufacturer_id_toyota,model_id_camry,model_id_civic,...,color_white,color_yellow,state_ca,state_fl,state_mn,state_ny,state_tx,state_va,state_wa,state_wi
18,1,2011.0,168955.0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
141,20988,2011.0,82000.0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
221,13699,2004.0,135101.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
520,20818,2013.0,50278.0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
642,4800,1997.0,111000.0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0


In [137]:
list(df.columns)

['customer_price',
 'year',
 'mileage',
 'manufacturer_id_chevrolet',
 'manufacturer_id_dodge',
 'manufacturer_id_ford',
 'manufacturer_id_honda',
 'manufacturer_id_toyota',
 'model_id_camry',
 'model_id_civic',
 'model_id_f-150',
 'model_id_silverado',
 'model_id_tacoma',
 'fuel_diesel',
 'fuel_gas',
 'transmission_automatic',
 'transmission_manual',
 'transmission_other',
 'type_hatchback',
 'type_pickup',
 'type_sedan',
 'type_truck',
 'type_van',
 'type_wagon',
 'color_black',
 'color_blue',
 'color_brown',
 'color_green',
 'color_grey',
 'color_orange',
 'color_purple',
 'color_red',
 'color_silver',
 'color_white',
 'color_yellow',
 'state_ca',
 'state_fl',
 'state_mn',
 'state_ny',
 'state_tx',
 'state_va',
 'state_wa',
 'state_wi']

In [138]:
df.shape

(5010, 43)

##### Model training 

In [162]:
y = df['customer_price']
X = df.drop('customer_price', axis = 1)

In [163]:
X.shape, y.shape

((439815, 2), (439815,))

In [164]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [165]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()

linreg.fit(X_train, y_train)

from sklearn.metrics import mean_absolute_error
# Evaluation on test dataset
y_pred_linreg = linreg.predict(X_test)
print(mean_absolute_error(y_test, y_pred_linreg))

283172.4613790094


In [166]:
# save the model to disk
import pickle
filename = 'linreg.sav'
pickle.dump(linreg, open(filename, 'wb'))

#### put model to google cloud

In [167]:
client = storage.Client()
bucket = client.get_bucket('car_platform_gcs_bucket')
blob = bucket.blob('models/linreg.sav')
blob.upload_from_filename('linreg.sav')

In [59]:
### tunning

In [168]:
skf5 = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)

from sklearn.linear_model import Lasso

parameters = {'alpha': [0.05, 0.1, 0.5, 0.75, 1]}
lassoreg = Lasso(normalize = True)
lassoreg_gcv = GridSearchCV(lassoreg, parameters, n_jobs = -1, cv = skf5, verbose = 1, 
                    scoring = 'neg_mean_absolute_error')
lassoreg_gcv.fit(X_train, y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


/Users/marichka/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    3.8s finished
/Users/marichka/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
       error_score='raise-deprecating',
       estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'alpha': [0.05, 0.1, 0.5, 0.75, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=1)

In [169]:
lassoreg_gcv.best_score_

-243436.1783071265

In [170]:
lassoreg_gcv.best_params_

{'alpha': 1}

In [171]:
# Initialize our classifier with default parameters
lassoreg = Lasso(normalize = True, alpha = 1)

# Evaluation on test dataset
y_pred_lassoreg = lassoreg.fit(X_train, y_train).predict(X_test)

In [172]:
# save the model to disk
import pickle
filename = 'lassoreg.sav'
pickle.dump(lassoreg, open(filename, 'wb'))

#### put model to google cloud

In [173]:
client = storage.Client()
bucket = client.get_bucket('car_platform_gcs_bucket')
blob = bucket.blob('models/lassoreg.sav')
blob.upload_from_filename('lassoreg.sav')

In [ ]:
##prediction

In [182]:
car_attributes = {'manufacturer_id': 'ford', 'model_id': 'f-150', 'fuel': 'gas', 
     'transmission': 'automatic', 'vin': '1FTFW1EF8EFA71429', 'mileage': 4000, 
     'year': 1990, 'type': 'sedan', 'color': 'gray'}
df = pd.DataFrame.from_dict(car_attributes, orient='index').T
df = df[['year', 'mileage']]

In [183]:
df.head()

,year,mileage
0,1990,4000


In [184]:
prediction = lassoreg.predict(df)